In [1]:
username = "Alex"
env = "integrate"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\" * env * "\\")
include(pathtorepo * "dynamical-systems\\system.jl")
include(pathtorepo * "dynamical-systems\\box_square_functions.jl")

using StaticArrays, DifferentialEquations, DynamicalSystems, JLD2
E, x, y  = -40..40, -5..5,  -5..5
box = E × x × y
using GLMakie, LinearAlgebra, LinearSolve

pathtofile = "C:\\Users\\" *username *  "\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\Сопоставление с матконт\\файлы matlab\\"
I0_hom = load(pathtofile * "I0_hom_hom.jld")["data"]
u0_hom = load(pathtofile * "U0_hom_hom.jld")["data"]
I0_hom = I0_hom[:]
u0_hom = u0_hom[:];

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`

In [2]:
index_point_from_curve = 1

const τ = 0.013;  const τD = 0.07993;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4; const α = 1.58; const ΔU0 = 0.305; const U0 = u0_hom[index_point_from_curve]

t = 2000.0; tspan = [0.0, t]
u0 = [0.0, 0.0, 0.0]
ϵ_box = 1.0e-5; ϵ_shift = 1.0e-2;

In [3]:
number_points_on_side = 10
total_count_points = number_points_on_side * 2

I0 = I0_hom[index_point_from_curve]

-1.70629651132633

In [6]:
p = SA[α, τ, τD, τy, J, xthr, ythr, U0, ΔU0, β, I0]

fixedpoint = get_fixed_point(TM, jacob_TM_, p, u0)
A = get_matrix(fixedpoint[1], p, jacob_TM_, 2)


condition = make_event(fixedpoint[1], ϵ_box, A)
cb = ContinuousCallback(condition, nothing, affect!)

ContinuousCallback{var"#condition#27"{SVector{3, Float64}, Float64, Matrix{Float64}}, Nothing, typeof(affect!), typeof(SciMLBase.INITIALIZE_DEFAULT), typeof(SciMLBase.FINALIZE_DEFAULT), Float64, Int64, Rational{Int64}, Nothing, Int64}(condition, nothing, affect!, SciMLBase.INITIALIZE_DEFAULT, SciMLBase.FINALIZE_DEFAULT, nothing, SciMLBase.LeftRootFind, 10, Bool[1, 1], 1, 2.220446049250313e-15, 0, 1//100)

In [14]:
u0 = SA[8.345810451419165, 0.7385044832615892, 0.43830030450649443]
prob = ODEProblem(TM, u0, tspan, p)
sol = solve(prob, alg = Vern9(), abstol = 1e-14, reltol = 1e-14, callback = cb)


retcode: Terminated
Interpolation: specialized 9th order lazy interpolation
t: 180268-element Vector{Float64}:
    0.0
    0.030920887027979592
    0.04966965313301733
    0.07089555414604806
    0.09091553496819131
    0.11033110356036858
    0.12951326368890145
    0.14821312270056533
    0.16696739501878988
    0.1861641487655803
    ⋮
 1928.9835923095534
 1929.0066649758612
 1929.03006884812
 1929.0523947856557
 1929.0748137895228
 1929.0990590377876
 1929.120981736987
 1929.1234228457663
 1929.1234228457663
u: 180268-element Vector{SVector{3, Float64}}:
 [8.345810451419165, 0.7385044832615892, 0.43830030450649443]
 [8.346277259208673, 0.7384975944292903, 0.43830058656308807]
 [8.346553905269584, 0.738492573486556, 0.43830058763649904]
 [8.34683049499266, 0.7384865740250764, 0.438300415368727]
 [8.347030825726161, 0.7384810529074972, 0.4383000825471508]
 [8.347148067651503, 0.7384762693783594, 0.43829961309245263]
 [8.347173947538563, 0.7384725043868261, 0.4382990301140898]
 [8.347

In [19]:
norm_, linsolve = get_norm_αs(Vector(sol[end]), Vector(fixedpoint[1]), A) 

(2.7813591103930512e-5, [1.0123496859795507e-5, 2.7813591103930512e-5, -6.45014459268911e-6])

In [6]:
number_of_event = length(findall(check_events .== true))
number_of_no_event = length(check_events) - number_of_event
println("number of event: $(number_of_event); number of no event: $(number_of_no_event)")

number of event: 4; number of no event: 16


In [7]:
αs

20×3 Matrix{Float64}:
 0.0         0.0          0.0
 0.0         0.0          0.0
 9.94549e-6  1.0e-5      -6.21704e-6
 0.0         0.0          0.0
 0.0         0.0          0.0
 1.01235e-5  2.78136e-5  -6.45014e-6
 0.0         0.0          0.0
 0.0         0.0          0.0
 0.0         0.0          0.0
 0.0         0.0          0.0
 0.0         0.0          0.0
 0.0         0.0          0.0
 0.0         0.0          0.0
 9.97805e-6  1.0e-5      -7.118e-6
 0.0         0.0          0.0
 0.0         0.0          0.0
 0.0         0.0          0.0
 0.0         0.0          0.0
 9.61955e-6  1.0e-5      -5.62205e-6
 0.0         0.0          0.0

In [18]:
αs_without_zeros =  permutedims(stack(filter(!iszero,eachrow(αs))))
# last points in space (α2, α3)
f = Figure(resolution = (700, 700))
ax = Axis(f[1, 1], xlabel = L"α_2", ylabel = L"α_3", xlabelsize = 35, ylabelsize = 35)
scatter!(ax, 0.0, 0.0, color = :green, markersize = 15, marker = :rect)

scatter!(ax, αs_without_zeros[:, 2], αs_without_zeros[:, 3], markersize = 5)

display(GLMakie.Screen(), f);

In [11]:
filename = "index_point_from_curve $(index_point_from_curve)"
filename = filename*".jld2"

"index_point_from_curve 1.jld2"

In [12]:
cd("C:\\Users\\" *username *  "\\Desktop\\dynamical-systems\\Tsodyks Markram\\Levanova\\3 набор параметров\\manifolds\\TM\\box square\\")
jldsave(filename; dots_u0, check_events, time_events, dots_on_event ,αs, norms)